In [ ]:
import pandas as pd
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = '/content/mf-abogados-0d430e6c401e.json'  #ruta del json archivo
VIEW_ID = '208483271'  # un numero de google analitics de esta web https://analytics.google.com/analytics/web/?authuser=2#/a155069538w218812185p208483271/admin/view/settings

def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)
  analytics = build('analyticsreporting', 'v4', credentials=credentials)
  return analytics

#Get one report page
def get_report(analytics, pageTokenVar):
  return analytics.reports().batchGet(
      body={

################################################################################## las dimensiones se ven estesta web https://ga-dev-tools.web.app/dimensions-metrics-explorer/

         'reportRequests': [
         {
           'viewId': VIEW_ID,
           'dateRanges': [{'startDate': '30daysAgo', 'endDate': 'yesterday'}],
           'metrics': [
               {'expression': 'ga:users'},
               {'expression': 'ga:newUsers'},
               {'expression': 'ga:pageviews'},
               {'expression': 'ga:bounces'},
               {'expression': 'ga:sessions'},
               {'expression': 'ga:goal1Completions'},
               {'expression': 'ga:timeOnPage'}
           ],
           'dimensions': [
               {'name': 'ga:date'},
               {'name': 'ga:sourceMedium'},
               {'name': 'ga:deviceCategory'},
               {'name': 'ga:city'},
               {'name': 'ga:country'},
               {'name': 'ga:sessionDurationBucket'},
               {'name': 'ga:landingPagePath'},
               {'name': 'ga:pagePath'}
           ],
           'samplingLevel': 'LARGE',
           'pageSize': 1000
         }]

##################################################################################
          
      }
  ).execute()
    
def handle_report(analytics,pagetoken,rows):  
    response = get_report(analytics, pagetoken)

    #Header, Dimentions Headers, Metric Headers 
    columnHeader = response.get("reports")[0].get('columnHeader', {})
    dimensionHeaders = columnHeader.get('dimensions', [])
    metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])

    #Pagination
    pagetoken = response.get("reports")[0].get('nextPageToken', None)
    
    #Rows
    rowsNew = response.get("reports")[0].get('data', {}).get('rows', [])
    rows = rows + rowsNew
    print("len(rows): " + str(len(rows)))

    #Recursivly query next page
    if pagetoken != None:
        return handle_report(analytics,pagetoken,rows)
    else:
        #nicer results
        nicerows=[]
        for row in rows:
            dic={}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dic[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dic[metric.get('name')] = float(value)
                    else:
                        dic[metric.get('name')] = float(value)
            nicerows.append(dic)
        return nicerows

#Start
def main():    
    analytics = initialize_analyticsreporting()
    
    global dfanalytics
    dfanalytics = []

    rows = []
    rows = handle_report(analytics,'0',rows)

    dfanalytics = pd.DataFrame(list(rows))

if __name__ == '__main__':
  main()

dfanalytics.head()


len(rows): 36


,ga:date,ga:sourceMedium,ga:deviceCategory,ga:city,ga:country,ga:sessionDurationBucket,ga:landingPagePath,ga:pagePath,ga:users,ga:newUsers,ga:pageviews,ga:bounces,ga:sessions,ga:goal1Completions,ga:timeOnPage
0,20220724,(direct) / (none),desktop,(not set),China,0,/,/,3.0,3.0,3.0,3.0,3.0,0.0,0.0
1,20220724,google / organic,mobile,(not set),Spain,658,/abogados-divorcios-tui/,/,1.0,0.0,2.0,0.0,0.0,0.0,61.0
2,20220724,google / organic,mobile,(not set),Spain,658,/abogados-divorcios-tui/,/abogados-divorcios-tui/,1.0,1.0,2.0,0.0,1.0,0.0,53.0
3,20220724,google / organic,mobile,(not set),Spain,658,/abogados-divorcios-tui/,/como-elegir-abogados-de-divorcios/,1.0,0.0,1.0,0.0,0.0,0.0,118.0
4,20220724,google / organic,mobile,(not set),Spain,658,/abogados-divorcios-tui/,/contacto/,1.0,0.0,2.0,0.0,0.0,1.0,32.0
